<img src="logo.png" alt="Competition Logo" width="200"/>

# ___TITLE___

Welcome to the competition! This notebook will help you get started. You'll learn how to:
1. Initialize and interact with the environment
2. Create and use a random agent as a baseline
3. Run local evaluation using `run_local.py`


## 1. Setup

First, let's install the required dependencies:


In [ ]:
%pip install -q -r requirements.txt


## 2. Initialize the Environment

Let's create and explore the environment:


In [ ]:
import gymnasium as gym
import numpy as np

# Environment ID for this competition
ENV_ID = "___ENV_ID___"

# Create the environment
env = gym.make(ENV_ID)

print(f"Environment: {ENV_ID}")
print(f"Observation Space: {env.observation_space}")
print(f"Action Space: {env.action_space}")


## 3. Render the Initial State

Let's reset the environment and try to render the initial state:


In [ ]:
import matplotlib.pyplot as plt

# Reset the environment to get the initial observation
observation, info = env.reset()

print(f"Initial Observation: {observation}")
print(f"Observation Shape: {observation.shape if hasattr(observation, 'shape') else type(observation)}")

# Try to render the environment (may not work in all environments)
try:
    env_render = gym.make(ENV_ID, render_mode="rgb_array")
    env_render.reset()
    frame = env_render.render()
    
    plt.figure(figsize=(8, 6))
    plt.imshow(frame)
    plt.title("Initial State")
    plt.axis('off')
    plt.show()
    env_render.close()
except Exception as e:
    print(f"Rendering not available: {e}")
    print("This is normal for some environments. You can still train and evaluate agents.")


## 4. Random Agent Baseline

Let's use the random agent as a baseline. This agent takes random actions and serves as a starting point:


In [ ]:
# Import and create the random agent
from random_agent import Agent

agent = Agent(env)
print(f"Agent created: {type(agent).__name__}")


## 5. Take a Random Action and Render

Let's use the agent to take a random action and see what happens:


In [ ]:
# Reset the environment
observation, info = env.reset()

# Get a random action from the agent
action = agent.act(observation)
print(f"Action taken: {action}")

# Take a step in the environment
next_observation, reward, terminated, truncated, info = env.step(action)

print(f"\nStep Result:")
print(f"  Reward: {reward}")
print(f"  Terminated: {terminated}")
print(f"  Truncated: {truncated}")
print(f"  Next Observation: {next_observation}")


In [ ]:
# Try to render the state after taking an action
try:
    env_render = gym.make(ENV_ID, render_mode="rgb_array")
    env_render.reset()
    env_render.step(action)
    frame = env_render.render()
    
    plt.figure(figsize=(8, 6))
    plt.imshow(frame)
    plt.title(f"State after action: {action}")
    plt.axis('off')
    plt.show()
    env_render.close()
except Exception as e:
    print(f"Rendering not available: {e}")


## 6. Run a Full Episode

Let's run a complete episode and track the total reward:


In [ ]:
def run_episode(env, agent, max_steps=1000):
    """Run a single episode and return the total reward."""
    observation, _ = env.reset()
    total_reward = 0
    steps = 0
    
    done = False
    while not done and steps < max_steps:
        action = agent.act(observation)
        observation, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        done = terminated or truncated
        steps += 1
    
    return total_reward, steps

# Run multiple episodes to get average performance
num_episodes = 10
rewards = []

for ep in range(num_episodes):
    reward, steps = run_episode(env, agent)
    rewards.append(reward)
    print(f"Episode {ep + 1}: Reward = {reward:.2f}, Steps = {steps}")

print(f"\n--- Random Agent Baseline ---")
print(f"Average Reward: {np.mean(rewards):.2f}")
print(f"Std Deviation: {np.std(rewards):.2f}")
print(f"Min/Max: {np.min(rewards):.2f} / {np.max(rewards):.2f}")


## 7. Using run_local.py for Evaluation

The `run_local.py` script allows you to test your agent implementation locally before submitting.

To use it, you need to:
1. Create an `agent.py` file with your `Agent` class
2. Run `python run_local.py` from the command line

Let's test this with the random agent:


In [ ]:
# Copy random_agent.py to agent.py so run_local.py can find it
import shutil
shutil.copy('random_agent.py', 'agent.py')
print("Copied random_agent.py to agent.py")


In [ ]:
# Run the local evaluation script
import subprocess
result = subprocess.run(['python', 'run_local.py'], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)


## 8. Next Steps

Now that you understand the basics, here's how to create your own agent:

1. **Start from the template**: Copy `random_agent.py` and rename it to `agent.py`

2. **Implement your agent**: Modify the `Agent` class to implement your RL algorithm:
   - `__init__(self, env)`: Initialize your model/network
   - `act(self, observation)`: Return an action given the current observation
   - `train(self)`: Train your agent on the environment
   - `load(self, path)`: Load saved model weights
   - `save(self, path)`: Save your trained model

3. **Test locally**: Use `run_local.py` to test your implementation

4. **Build a submission**: Use `build_submission.py` to create the zip file (see below)

5. **Submit**: Upload the zip to the competition page

Good luck! 🚀


## 9. Building a Submission

The `build_submission.py` script creates a zip file ready for submission. It zips the **contents** of a folder (not the folder itself), which is what Codabench expects.

**From the command line** (run from the starting kit directory):

```bash
# Default: zips sample_submission/ → sample_submission.zip
python build_submission.py

# Custom input and output
python build_submission.py -i sample_submission -o my_submission.zip
```

**Options:**
- `-i`, `--input`: Folder to zip (default: `sample_submission`)
- `-o`, `--output`: Output zip path (default: `sample_submission.zip`)

Place your `agent.py`, any model files (`model.pt`, etc.) and extra artifacts needed (e.g. `requirements.txt`) in the folder you zip, then upload the resulting zip to the competition.

In [ ]:
# Clean up
env.close()
